# Preprocessing Framework

Per prima cosa importiamo le librerie che utilizzeremo nel progetto. 

In [1]:
import numpy as np 
import pandas as pd

Una volte che abbiamo scaricato il dataset creiamo il DataFrame *pandas* dove salvare i dati. 

In [4]:
# dipendenza: `openpyxl`
path = '/home/carlo-bianchi/Documents/Coding/PythonForDataAnanlysis/examples/gamezone-orders-data.xlsx'
data = pd.read_excel(path)
data.head()

,USER_ID,ORDER_ID,PURCHASE_TS,SHIP_TS,PRODUCT_NAME,PRODUCT_ID,USD_PRICE,PURCHASE_PLATFORM,MARKETING_CHANNEL,ACCOUNT_CREATION_METHOD,COUNTRY_CODE
0,2c06175e,0001328c3c220830,2020-12-24 00:00:00,2020-12-13,Nintendo Switch,e682,168.00,website,affiliate,unknown,US
1,ee8e5bc2,0002af7a5c6100772,2020-10-01 00:00:00,2020-09-21,Nintendo Switch,e682,160.61,website,direct,desktop,DE
2,9eb4efe0,0002b8350e167074,2020-04-21 00:00:00,2020-02-16,Nintendo Switch,8d0d,151.20,website,direct,desktop,US
3,cac7cbaf,0006d06b98385729,2020-04-07 00:00:00,2020-04-04,Sony PlayStation 5 Bundle,54ed,1132.82,website,direct,desktop,AU
4,6b0230bc,00097279a2f46150,2020-11-24 00:00:00,2020-08-02,Nintendo Switch,8d0d,33.89,website,direct,desktop,TR


Diamo un'occhiata a:
- i tipi di dati del nostro dataset
- la numerosita del dataset 

In [83]:
data.shape

(21864, 11)

In [84]:
data.dtypes

USER_ID                            object
ORDER_ID                           object
PURCHASE_TS                        object
SHIP_TS                    datetime64[ns]
PRODUCT_NAME                       object
PRODUCT_ID                         object
USD_PRICE                         float64
PURCHASE_PLATFORM                  object
MARKETING_CHANNEL                  object
ACCOUNT_CREATION_METHOD            object
COUNTRY_CODE                       object
dtype: object

### Il Framework CLEAN

<center>
<img src="./images/CLEAR_framework.jpg" width="500" >
</center>

**1. C - Conceptualize (Concettualizzare i dati)**
Prima di toccare qualsiasi dato, è fondamentale capire cosa si ha davanti. L'analista deve identificare tre elementi chiave:
* **La "Grana" (Grain):** Cosa rappresenta ogni riga? In questo esempio, ogni riga è un ordine unico.
* **Metriche Chiave:** I numeri importanti (es. prezzo in USD).
* **Dimensioni Chiave:** Gli attributi per raggruppare i dati (es. tempo, nome prodotto, canale di marketing, paese).
Questo passaggio aiuta a capire quali colonne sono prioritarie per rispondere alle domande con le quali interrogari i dati.

**2. L - Locate Solvable Issues (Individuare problemi risolvibili)**
Questa fase riguarda la correzione di errori evidenti che non richiedono input esterni. Si crea un **"Issues Log"** (registro dei problemi) per tracciare tutto.
* **Tipi di problemi:** Formati data inconsistenti, errori di ortografia, categorie nulle che possono essere etichettate come "Sconosciuto" (Unknown).
* **Best Practice:** Non sovrascrivere mai i dati originali. Crea una copia o una nuova colonna (es. `date_cleaned`) finché non sei sicuro della modifica.

**3. E - Evaluate Unsolvable Issues (Valutare problemi non risolvibili)**
Qui si affrontano problemi come dati mancanti (senza fonte di verità), outlier o violazioni della logica di business (es. data di spedizione antecedente alla data di acquisto).
* **Euristica:** Se l'impatto del problema (magnitudine) è basso (es. meno del 20% dei dati), spesso è meglio documentarlo e lasciarlo così piuttosto che cancellare i dati.
* **Imputazione:** Gli analisti dovrebbero evitare di "inventare" dati (imputazione) a meno che non ci sia una logica di business chiarissima, per non introdurre bias. Per quanto riguarda un Data Scientist, dipende da come i dati debbano essere impiegati.

**4. A - Augment the Data (Aumentare/Arricchire i dati)**
Questa fase serve a rendere il dataset più flessibile e potente per l'analisi.
* **Cosa fare:** Creare nuove colonne utili. Ad esempio, scomporre una data in colonne separate per Anno, Mese e Settimana per facilitare i filtri temporali.
* **Nuove Metriche:** Calcolare metriche derivate, come il "tempo di spedizione" (differenza in giorni tra data spedizione e acquisto).

**5. N - Note and Document (Annotare e Documentare)**
L'ultimo passo è finalizzare l'Issues Log.
* Bisogna annotare non solo il problema, ma anche la **magnitudine** (quanti dati sono impattati in percentuale) e la **decisione finale** presa (es. "Risolto", "Lasciato così", "Documentato").
* Questo serve come "paper trail" (traccia scritta) per dimostrare trasparenza e professionalità agli stakeholder o ai manager che revisionano il progetto.


### 1. 🇨 - Conceptualize (Concettualizzazione)

Fase preliminare di comprensione del dataset (nessuna modifica ai dati).

- Identificare la "Grana": Verificare che ogni riga corrisponda a un ordine unico (order_id).

- Identificare le Metriche: Individuare la colonna numerica chiave (usd_price).

- Identificare le Dimensioni: Individuare le colonne categoriche principali (`purchase_timestamp`, `product_name`, `marketing_channel`, `country_code`, ecc.).

In [85]:
data.head()

,USER_ID,ORDER_ID,PURCHASE_TS,SHIP_TS,PRODUCT_NAME,PRODUCT_ID,USD_PRICE,PURCHASE_PLATFORM,MARKETING_CHANNEL,ACCOUNT_CREATION_METHOD,COUNTRY_CODE
0,2c06175e,0001328c3c220830,2020-12-24 00:00:00,2020-12-13,Nintendo Switch,e682,168.00,website,affiliate,unknown,US
1,ee8e5bc2,0002af7a5c6100772,2020-10-01 00:00:00,2020-09-21,Nintendo Switch,e682,160.61,website,direct,desktop,DE
2,9eb4efe0,0002b8350e167074,2020-04-21 00:00:00,2020-02-16,Nintendo Switch,8d0d,151.20,website,direct,desktop,US
3,cac7cbaf,0006d06b98385729,2020-04-07 00:00:00,2020-04-04,Sony PlayStation 5 Bundle,54ed,1132.82,website,direct,desktop,AU
4,6b0230bc,00097279a2f46150,2020-11-24 00:00:00,2020-08-02,Nintendo Switch,8d0d,33.89,website,direct,desktop,TR


In [88]:
# informazioni globali sulla nostra tabella 
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21864 entries, 0 to 21863
Data columns (total 11 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   USER_ID                  21864 non-null  object        
 1   ORDER_ID                 21864 non-null  object        
 2   PURCHASE_TS              21864 non-null  object        
 3   SHIP_TS                  21864 non-null  datetime64[ns]
 4   PRODUCT_NAME             21864 non-null  object        
 5   PRODUCT_ID               21864 non-null  object        
 6   USD_PRICE                21859 non-null  float64       
 7   PURCHASE_PLATFORM        21864 non-null  object        
 8   MARKETING_CHANNEL        21781 non-null  object        
 9   ACCOUNT_CREATION_METHOD  21781 non-null  object        
 10  COUNTRY_CODE             21826 non-null  object        
dtypes: datetime64[ns](1), float64(1), object(9)
memory usage: 1.8+ MB


In [87]:
# descrizione delle metriche chiave 
data['USD_PRICE'].describe()

count    21859.000000
mean       281.406583
std        366.296410
min          0.000000
25%        126.000000
50%        168.000000
75%        357.000000
max       3146.880000
Name: USD_PRICE, dtype: float64

### 2. 🇱 - Locate Solvable Issues (Problemi Risolvibili)
*Pulizia attiva degli errori evidenti e standardizzazione.*
- Tipizzazione corretta delle colonne: 
  - dobbiamo ancora assegnare il tipo giusto ad ognuno dei campi del dataset
  - **Azione Pandas**: usare i metodi `convert_dtypes()` e `astype()`
- Standardizzazione Date:
    - La colonna `purchase_timestamp` ha formati misti,
    - **Azione Pandas**: Convertire la colonna in formato datetime (gestendo i formati misti) per uniformare tutto con `to_datetime()`.

- Pulizia Stringhe (Product Name):
    - Ci sono inconsistenze come "27in" vs "27 in".
    - **Azione Pandas**: Standardizzare i valori nella colonna `product_name` (es. tramite un dizionario di mapping o replace) con `replace()`.

- Gestione Valori Mancanti (Categorie):
    - Le colonne `marketing_channel` e `account_creation_method` hanno valori vuoti (`null`).
    - **Azione Pandas**: Riempire i valori `NaN` con la stringa `'Unknown'` usando `fillna()`.
- Analisi Duplicati:
    - Controllo se ci sono `order_id` duplicati associati a `user_id` diversi.
    - **Azione Pandas**: Identificare i duplicati. Decidere come trattarli, usando `duplicated()`.

In [90]:
# copia dei dati che puliremo 
data_copy = data.copy()

#### Formattazione dei tipi delle colonne

Per prima cosa dobbiamo trovare il giusto tipe pe ogni campo del dataset. Possiamo utilizzare un metodo automatico di *pandas*, che permette, utilizzando le stesse regole inferenziali per il tipo dell'inizializzatore dell'oggetto `DataFrame`, di assegnare il miglior tipo di dato ai campi del dataset. 

In [123]:
data_copy = data_copy.convert_dtypes()

In [125]:
data_copy.dtypes

USER_ID                            object
ORDER_ID                   string[python]
PURCHASE_TS                datetime64[ns]
SHIP_TS                    datetime64[ns]
PRODUCT_NAME               string[python]
PRODUCT_ID                         object
USD_PRICE                         Float64
PURCHASE_PLATFORM          string[python]
MARKETING_CHANNEL          string[python]
ACCOUNT_CREATION_METHOD    string[python]
COUNTRY_CODE               string[python]
dtype: object

Adesso mancano solamente i campi USER_ID e PRODUCT_ID. Cerchiamo di capire come fare. Probabilmente le regole di inferenza automatiche di *pandas* per il tipo, non hanno saputo come intervenire con queste colonne. 

Tuttavia, guardando al tipo di dato a mano, possiamo vedere come quei campi siano popolati da stringhe.

In [143]:
data_copy['USER_ID'] = data_copy['USER_ID'].astype('string')
data_copy['PRODUCT_ID'] = data_copy['PRODUCT_ID'].astype('string')
data_copy.dtypes

USER_ID                    string[python]
ORDER_ID                   string[python]
PURCHASE_TS                datetime64[ns]
SHIP_TS                    datetime64[ns]
PRODUCT_NAME               string[python]
PRODUCT_ID                 string[python]
USD_PRICE                         Float64
PURCHASE_PLATFORM          string[python]
MARKETING_CHANNEL          string[python]
ACCOUNT_CREATION_METHOD    string[python]
COUNTRY_CODE               string[python]
dtype: object

#### Standardizzazione delle date 

In [144]:
data_copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21864 entries, 0 to 21863
Data columns (total 11 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   USER_ID                  21864 non-null  string        
 1   ORDER_ID                 21864 non-null  string        
 2   PURCHASE_TS              21853 non-null  datetime64[ns]
 3   SHIP_TS                  21864 non-null  datetime64[ns]
 4   PRODUCT_NAME             21864 non-null  string        
 5   PRODUCT_ID               21864 non-null  string        
 6   USD_PRICE                21859 non-null  Float64       
 7   PURCHASE_PLATFORM        21864 non-null  string        
 8   MARKETING_CHANNEL        21781 non-null  string        
 9   ACCOUNT_CREATION_METHOD  21781 non-null  string        
 10  COUNTRY_CODE             21826 non-null  string        
dtypes: Float64(1), datetime64[ns](2), string(8)
memory usage: 1.9 MB


Nel caso delle date, dobbiamo fare piu attenzione. Nonostante automatiamente *pandas* abbia riconosciuto il giusto tipo, potremo voler specificare una formattazione preferita per i dati. Inoltre, nel caso in cui ci siano degli errori nella data, vorremo sostituire questi valori con il simbolo `NaT` ('Not a Time).  

In [ ]:
# formattazione delle colonne con date, coerce in caso di data errata, sostituisce con NaT 
data_copy['PURCHASE_TS'] = pd.to_datetime(data_copy['PURCHASE_TS'], errors='coerce')
data_copy['SHIP_TS'] = pd.to_datetime(data_copy['SHIP_TS'], errors='coerce')

Possiamo notare come le date adesso appaiano nel formato desiderato: 'YYYY-MM-DD'.

In [96]:
data_copy.head()

,USER_ID,ORDER_ID,PURCHASE_TS,SHIP_TS,PRODUCT_NAME,PRODUCT_ID,USD_PRICE,PURCHASE_PLATFORM,MARKETING_CHANNEL,ACCOUNT_CREATION_METHOD,COUNTRY_CODE
0,2c06175e,0001328c3c220830,2020-12-24,2020-12-13,Nintendo Switch,e682,168.00,website,affiliate,unknown,US
1,ee8e5bc2,0002af7a5c6100772,2020-10-01,2020-09-21,Nintendo Switch,e682,160.61,website,direct,desktop,DE
2,9eb4efe0,0002b8350e167074,2020-04-21,2020-02-16,Nintendo Switch,8d0d,151.20,website,direct,desktop,US
3,cac7cbaf,0006d06b98385729,2020-04-07,2020-04-04,Sony PlayStation 5 Bundle,54ed,1132.82,website,direct,desktop,AU
4,6b0230bc,00097279a2f46150,2020-11-24,2020-08-02,Nintendo Switch,8d0d,33.89,website,direct,desktop,TR


Per concludere con i campi di date, non rimane che controllare se ci sono dei dati mancanti. 

In [147]:
data_copy[data_copy['SHIP_TS'].isna()].shape

(0, 11)

In [148]:
data_copy[data_copy['PURCHASE_TS'].isna()]

,USER_ID,ORDER_ID,PURCHASE_TS,SHIP_TS,PRODUCT_NAME,PRODUCT_ID,USD_PRICE,PURCHASE_PLATFORM,MARKETING_CHANNEL,ACCOUNT_CREATION_METHOD,COUNTRY_CODE
1047,a5298a4d,0dda212aaea69940,NaT,2019-07-08,JBL Quantum 100 Gaming Headset,ab0f,21.96,website,direct,desktop,FR
1251,e749986a,1041360a9a997694,NaT,2020-11-03,27in 4k Gaming Monitor,891b,480.0,website,direct,desktop,US
5846,a81bb521,4cd9ab100d971208,NaT,2021-01-11,Nintendo Switch,8d0d,120.26,website,direct,desktop,IE
7363,7c1b4739,60d6f8dd5dc53016,NaT,2020-06-15,JBL Quantum 100 Gaming Headset,ab0f,21.46,website,direct,desktop,ES
9717,1db9d78a,7ed14bf67eb15182,NaT,2021-01-03,Sony PlayStation 5 Bundle,54ed,1800.0,website,direct,desktop,US
10011,81d147e3,829839cc52155359,NaT,2021-11-16,JBL Quantum 100 Gaming Headset,4db1,14.29,mobile app,direct,desktop,QA
10890,6dcd78e9,8d6795b3b1447377,NaT,2020-02-20,27in 4k Gaming Monitor,891b,408.0,website,direct,mobile,US
11853,2fa9f33d,99d824517da22388,NaT,2019-04-11,JBL Quantum 100 Gaming Headset,ab0f,21.19,website,direct,mobile,JP
14576,43bbad06,ba7f92bd22730500,NaT,2019-03-31,JBL Quantum 100 Gaming Headset,ab0f,24.0,website,direct,desktop,US
16163,b313cea5,c9e0aea0d9a75871,NaT,2019-05-18,JBL Quantum 100 Gaming Headset,ab0f,19.2,website,direct,desktop,US


Notiamo come manchino dei dati nel campo `PURCHASE_TS`. Scriviamo una fuzione che utilizzeremo anche in seguito, per calcolare la quantita di righe del dataset che soffrono di questo problema.

In [225]:
def percent(data):
    return len(data)/len(data_copy)

In [226]:
missing_pts = percent(data_copy[data_copy['PURCHASE_TS'].isna()])
print(missing_pts)

0.0005031101353823637


#### Formattazione dei nomi dei prodotti

In [100]:
data_copy.head()

,USER_ID,ORDER_ID,PURCHASE_TS,SHIP_TS,PRODUCT_NAME,PRODUCT_ID,USD_PRICE,PURCHASE_PLATFORM,MARKETING_CHANNEL,ACCOUNT_CREATION_METHOD,COUNTRY_CODE
0,2c06175e,0001328c3c220830,2020-12-24,2020-12-13,Nintendo Switch,e682,168.00,website,affiliate,unknown,US
1,ee8e5bc2,0002af7a5c6100772,2020-10-01,2020-09-21,Nintendo Switch,e682,160.61,website,direct,desktop,DE
2,9eb4efe0,0002b8350e167074,2020-04-21,2020-02-16,Nintendo Switch,8d0d,151.20,website,direct,desktop,US
3,cac7cbaf,0006d06b98385729,2020-04-07,2020-04-04,Sony PlayStation 5 Bundle,54ed,1132.82,website,direct,desktop,AU
4,6b0230bc,00097279a2f46150,2020-11-24,2020-08-02,Nintendo Switch,8d0d,33.89,website,direct,desktop,TR


Guardando i nomi dei prodotti che e possibile acquistare sul sito, si nota un problema. I due nomi: `27in 4K gaming monitor` e `27inches 4k gaming monitor` fanno riferimento allo stesso prodotto. Tuttavia c'e un errore di battitua che possiamo correggere. 

In [101]:
data_copy['PRODUCT_NAME'].unique()

array(['Nintendo Switch', 'Sony PlayStation 5 Bundle',
       '27in 4K gaming monitor', 'JBL Quantum 100 Gaming Headset',
       'Dell Gaming Mouse', 'Acer Nitro V Gaming Laptop',
       'Lenovo IdeaPad Gaming 3', 'Razer Pro Gaming Headset',
       '27inches 4k gaming monitor'], dtype=object)

In [102]:
# operazione di replace tramite dizionario di mapping
product_mapping = {
    '27in 4K gaming monitor': '27in 4k Gaming Monitor',
    '27inches 4k gaming monitor': '27in 4k Gaming Monitor',
}

data_copy['PRODUCT_NAME'] = data_copy['PRODUCT_NAME'].replace(product_mapping)

In [103]:
# correzione avvenuta:
data_copy['PRODUCT_NAME'].unique()

array(['Nintendo Switch', 'Sony PlayStation 5 Bundle',
       '27in 4k Gaming Monitor', 'JBL Quantum 100 Gaming Headset',
       'Dell Gaming Mouse', 'Acer Nitro V Gaming Laptop',
       'Lenovo IdeaPad Gaming 3', 'Razer Pro Gaming Headset'],
      dtype=object)

#### Valori Mancanti in ogni campo 

##### PRODUCT_ID

In [149]:
data_copy['PRODUCT_ID'].unique() # tutto ok 

<StringArray>
['e682', '8d0d', '54ed', '891b', 'ab0f', 'e7e6', '2997', 'b5f7', '8e5d',
 '8315', 'f81e', '5142', '22ea', '9ef0', '0c5a', '8d4f', '04ac', '1238',
 'b0ee', '7416', 'a6be', '2a50', 'ab5d', '7d63', 'df85', '0d23', '604c',
 '6b8d', '7f86', 'afbf', 'f5ca', '4c58', '4db1', '03ca', 'da12', 'e22d',
 '640d', '7599', '8364', '97c6', '24c1', '7388', 'ae96', '12b1', 'f443',
 '4f26']
Length: 46, dtype: string

##### USD_PRICE

Se ordiniamo l'array dei valori assunti nel campo USD_PRICE vediamo come ci siano 2 problemi diversi in questa colonna:
- il valore 0
- il valore *nan*

Affrontiamo questi problemi unno per volta. 

In [239]:
np.sort(data_copy['USD_PRICE'].unique())

array([   0.  ,    6.11,    6.32, ..., 2509.67, 3146.88,     nan],
      shape=(3634,))

In [150]:
data_copy[data_copy['USD_PRICE'].isna()]

,USER_ID,ORDER_ID,PURCHASE_TS,SHIP_TS,PRODUCT_NAME,PRODUCT_ID,USD_PRICE,PURCHASE_PLATFORM,MARKETING_CHANNEL,ACCOUNT_CREATION_METHOD,COUNTRY_CODE
1190,f2939b5c,0fa89eecc5d102514,2020-08-20,2020-05-26,Dell Gaming Mouse,0c5a,<NA>,mobile app,email,desktop,GH
13282,45913313,ab0b1a9c91a1197,2019-12-25,2019-12-28,JBL Quantum 100 Gaming Headset,2997,<NA>,mobile app,email,mobile,KE
14189,1c173578,b6a132d331c14466,2020-09-01,2020-09-04,Dell Gaming Mouse,f81e,<NA>,mobile app,direct,desktop,KE
20044,7c364c50,eee5663869153075,2020-07-11,2020-07-12,Dell Gaming Mouse,f81e,<NA>,mobile app,email,mobile,VE
20227,70565cdf,f063ed5ec8848400,2021-01-08,2021-01-10,Dell Gaming Mouse,f81e,<NA>,mobile app,email,desktop,BO


Possiamo imputare il valore che dobbiamo sostituire in queste posizioni in base al prodotto acquistato, specificato nella relativa colonna. Indagando, ci rendiamo conto come questa, nonostante fosse una buona idea, non puo essere applicata. Infatti, a seconda della transazione, il prezzo cambia di volta in volta. 

In [162]:
data_copy['USD_PRICE'][data_copy['PRODUCT_NAME'] == 'Dell Gaming Mouse'].unique()

<FloatingArray>
[50.09, 49.98, 46.58, 57.07, 64.25, 55.14, 57.55, 50.68, 53.09, 53.75,
 ...
 53.16, 37.61,  48.5, 49.53, 50.73, 49.63, 62.59, 51.19, 52.26, 38.65]
Length: 189, dtype: Float64

Un altro fattore da verificare e la presenza di transazione a costo 0. Questo tipo di transazioni rappresenta un errore all'interno dei nostri dati, nessuna transazione puo essere venuta senza che un presso sia stato pagato. 

In [172]:
0 in data_copy['USD_PRICE'].unique()

True

In [234]:
data_copy['USD_PRICE'].unique().min()

np.float64(0.0)

In [177]:
data_copy[data_copy['USD_PRICE'] == 0].shape

(29, 11)

Non avendo altre fonti per imputare i nostri dati, decidiamo di lasciar stare queste righe. Calcoliamo quanto questi problemi siano rilevanti in percentuale della numerosita del campione che abbiamo. 

In [240]:
missing_usd_na = percent(data_copy[data_copy['USD_PRICE'].isna()])
print(missing_usd_na)

missing_usd_zero = percent(data_copy[data_copy['USD_PRICE'] == 0])
print(missing_usd_zero)

0.00022868642517380168
0.0013263812660080497


Definiamo una funzione apposita per snellire il lavoro che faremo sui restanti campi. 

In [178]:
def df_isna(df, col):
    print(df[col].unique())
    return df[col][df[col].isna()]

##### PURCHASE_PLATFORM

In [179]:
df_isna(data_copy, 'PURCHASE_PLATFORM')

<StringArray>
['website', 'mobile app']
Length: 2, dtype: string


Series([], Name: PURCHASE_PLATFORM, dtype: string)

##### MARKETING_CHANNEL, ACCOUNT_CREATION_METHOD

Notiamo come manchino 83 valori per il campo MARKETING_CHANNEL e ACCOUNT_CREATION_METHOD. 

Per quanto riguarda il campo ACCOUNT_CREATION_METHOD non possiamo fare niente, in mancanza delle risorse necessarie per poter sostituire i valori mancanti. Per la colonna MARKETING_CHANNEL possiamo sostituire i valori nulli con `'unknown'`. 

In [180]:
df_isna(data_copy, 'MARKETING_CHANNEL')

<StringArray>
['affiliate', 'direct', 'email', 'social media', <NA>, 'unknown']
Length: 6, dtype: string


243      <NA>
563      <NA>
783      <NA>
1495     <NA>
1642     <NA>
         ... 
19854    <NA>
19929    <NA>
20114    <NA>
20277    <NA>
20999    <NA>
Name: MARKETING_CHANNEL, Length: 83, dtype: string

In [244]:
missing_markchan = 83/len(data_copy)
print(missing_markchan)

0.003796194657885108


Guardando i dati, si nota come sia presente il valore `unknown` nel campo MARKETING_CHANNEL. Sostituiamo i valori mancanti (`<NA>`) con questo valore. 

In [185]:
# sostituzione con unknown
data_copy['MARKETING_CHANNEL'] = data_copy['MARKETING_CHANNEL'].fillna('unknown')
df_isna(data_copy, 'MARKETING_CHANNEL')

<StringArray>
['affiliate', 'direct', 'email', 'social media', 'unknown']
Length: 5, dtype: string


Series([], Name: MARKETING_CHANNEL, dtype: string)

Per quanto riguarda il campo ACCOUNT_CREATION_METHOD, di nuovo, ci troviamo a non avere le fonte necessarie per poter sostituire i valori mancanti in maniera informata. 

In [186]:
df_isna(data_copy, 'ACCOUNT_CREATION_METHOD')

<StringArray>
['unknown', 'desktop', 'mobile', 'tablet', <NA>, 'tv']
Length: 6, dtype: string


243      <NA>
563      <NA>
783      <NA>
1495     <NA>
1642     <NA>
         ... 
19854    <NA>
19929    <NA>
20114    <NA>
20277    <NA>
20999    <NA>
Name: ACCOUNT_CREATION_METHOD, Length: 83, dtype: string

In [245]:
missing_acc_cr = 83/len(data_copy)
print(missing_acc_cr)

0.003796194657885108


#### Valori duplicati 

In [203]:
data_copy.head()

,USER_ID,ORDER_ID,PURCHASE_TS,SHIP_TS,PRODUCT_NAME,PRODUCT_ID,USD_PRICE,PURCHASE_PLATFORM,MARKETING_CHANNEL,ACCOUNT_CREATION_METHOD,COUNTRY_CODE
0,2c06175e,0001328c3c220830,2020-12-24,2020-12-13,Nintendo Switch,e682,168.0,website,affiliate,unknown,US
1,ee8e5bc2,0002af7a5c6100772,2020-10-01,2020-09-21,Nintendo Switch,e682,160.61,website,direct,desktop,DE
2,9eb4efe0,0002b8350e167074,2020-04-21,2020-02-16,Nintendo Switch,8d0d,151.2,website,direct,desktop,US
3,cac7cbaf,0006d06b98385729,2020-04-07,2020-04-04,Sony PlayStation 5 Bundle,54ed,1132.82,website,direct,desktop,AU
4,6b0230bc,00097279a2f46150,2020-11-24,2020-08-02,Nintendo Switch,8d0d,33.89,website,direct,desktop,TR


Controllo se ci sono `order_id` duplicati associati a `user_id` diversi

In [ ]:
# verifica
dupl = data_copy.duplicated(['ORDER_ID']) # maschera per i valori duplicati 
data_dupl_ord = data_copy[['USER_ID', 'ORDER_ID']][dupl] # sotto-frame dei valori duplicati  
len(data_dupl_ord['USER_ID'].unique()) == len(data_dupl_ord['ORDER_ID'].unique()) # verifica della richiesta

False

Utilizziamo la funzione `nunique()` che permette di contare il numero di elementi unici all'interno del dataset:

In [248]:
n_unique_val = data_copy['USER_ID'].nunique()
not_unique = len(data_copy) - data_copy['USER_ID'].nunique()
print(not_unique)

2013


In [249]:
n_duplicate = not_unique/len(data_copy)
print(n_duplicate)

0.09206915477497256


Adesso dobbiamo individuare quali sono le righe che hanno questo problema. 

Il codice che segue:
1. raggruppa i dati per ORDER_ID in *sotto-DataFrame*,
2. scarte i gruppi nei quali ci sono piu di due USER_ID diversi. 

In [217]:
conflicts = data_copy.groupby('ORDER_ID').filter(lambda x: x['USER_ID'].nunique() > 1)

print(conflicts)

        USER_ID           ORDER_ID PURCHASE_TS    SHIP_TS     PRODUCT_NAME  \
18909  52e2062d   e44ebe4eed936564  2021-02-10 2021-02-11  Nintendo Switch   
18910   7d98183   e44ebe4eed936564  2021-02-10 2021-02-11  Nintendo Switch   
18915  18d41108   e45770836b313082  2021-02-18 2021-02-21  Nintendo Switch   
18916   d415794   e45770836b313082  2021-02-18 2021-02-21  Nintendo Switch   
18986  ee4b40dd  e4f2373fbcb100655  2021-02-25 2021-02-26  Nintendo Switch   
...         ...                ...         ...        ...              ...   
21735   b0a2896   fecc9d6766231653  2021-02-22 2021-02-25  Nintendo Switch   
21742  e69c1f1e   fede5c24b0797422  2021-02-23 2021-02-25  Nintendo Switch   
21743   9c12903   fede5c24b0797422  2021-02-23 2021-02-25  Nintendo Switch   
21751  ebf6f56d   fef061c2a1499684  2021-02-06 2021-02-09  Nintendo Switch   
21752   f6f3685   fef061c2a1499684  2021-02-06 2021-02-09  Nintendo Switch   

      PRODUCT_ID  USD_PRICE PURCHASE_PLATFORM MARKETING_CHANNEL

Nonostante questo problema colpisca il 10% dei nostri dati, non abbiamo modo di risolverlo, in mancanza di una fonte di dati ufficiale che ci aiuti a correggere i dati corrotti. 

#### Problemi di Logica di Spedizione 

Infine notiamo come ci siano delle entrate in cui SHIP_TS e minore di PURCHASE_TS. Anche in questo caso il meglio che possiamo fare e additare l'errore. Non abbiamo fonti valide per sostituire i dati. 

In [252]:
invalid_shippig_logic = data_copy['SHIP_TS'] < data_copy['PURCHASE_TS']
num_invalid_logic = invalid_shippig_logic.sum()/len(data_copy)
print(num_invalid_logic)

0.09115440907427735


### 3. 🇪 - Evaluate Unsolvable Issues (Valutare problemi non risolvibili)

In [196]:
print("\n--- STEP E: Evaluate Unsolvable Issues ---")

# E1. Controllo Prezzi a 0 o Date Mancanti
missing_dates_count = data_copy['PURCHASE_TS'].isna().sum()
zero_price_count = (data_copy['USD_PRICE'] == 0).sum()

print(f"Ordini con data mancante: {missing_dates_count}")
print(f"Ordini con prezzo 0: {zero_price_count}")

# E2. Logica di Business (Spedizione prima dell'acquisto)
# Creiamo una maschera booleana
invalid_shipping_logic = data_copy['SHIP_TS'] < data_copy['PURCHASE_TS']
num_invalid_logic = invalid_shipping_logic.sum()
pct_invalid = (num_invalid_logic / len(data_copy)) * 100

print(f"Ordini con logica spedizione errata: {num_invalid_logic} ({pct_invalid:.1f}%)")

# DECISIONE: Nel video decide di NON eliminare queste righe, ma di annotarle.
# Se volessi eliminarle faresti: df = df[~invalid_shipping_logic]


--- STEP E: Evaluate Unsolvable Issues ---
Ordini con data mancante: 11
Ordini con prezzo 0: 29
Ordini con logica spedizione errata: 1993 (9.1%)


### 4. 🅰️ - Augment Data (Arricchire i dati)

In [262]:
print("\n--- STEP A: Augment Data ---")

# A1. Estrazione Granularità Temporale
data_copy['purchase_year'] =    data_copy['PURCHASE_TS'].dt.year
data_copy['purchase_month'] = data_copy['PURCHASE_TS'].dt.month_name()

# A2. Nuova Metrica: Time to Ship (Giorni tra acquisto e spedizione)
data_copy['time_to_ship_days'] = (data_copy['PURCHASE_TS'] - data_copy['PURCHASE_TS']).dt.days

data_copy.head()


--- STEP A: Augment Data ---


,USER_ID,ORDER_ID,PURCHASE_TS,SHIP_TS,PRODUCT_NAME,PRODUCT_ID,USD_PRICE,PURCHASE_PLATFORM,MARKETING_CHANNEL,ACCOUNT_CREATION_METHOD,COUNTRY_CODE,purchase_year,purchase_month,time_to_ship_days
0,2c06175e,0001328c3c220830,2020-12-24,2020-12-13,Nintendo Switch,e682,168.0,website,affiliate,unknown,US,2020.0,December,0.0
1,ee8e5bc2,0002af7a5c6100772,2020-10-01,2020-09-21,Nintendo Switch,e682,160.61,website,direct,desktop,DE,2020.0,October,0.0
2,9eb4efe0,0002b8350e167074,2020-04-21,2020-02-16,Nintendo Switch,8d0d,151.2,website,direct,desktop,US,2020.0,April,0.0
3,cac7cbaf,0006d06b98385729,2020-04-07,2020-04-04,Sony PlayStation 5 Bundle,54ed,1132.82,website,direct,desktop,AU,2020.0,April,0.0
4,6b0230bc,00097279a2f46150,2020-11-24,2020-08-02,Nintendo Switch,8d0d,33.89,website,direct,desktop,TR,2020.0,November,0.0


### 5. 🇳 - Note and Document (Annotare)

In [200]:
print("\n--- STEP N: Note & Document ---")

issues_log = [
    {
        'Issue': 'Formati data inconsistenti',
        'Status': 'Solved',
        'Action': 'Convertito a datetime ISO 8601',
        'Impact': '100% rows'
    },
    {
        'Issue': 'Nomi prodotti errati (typos)',
        'Status': 'Solved',
        'Action': 'Standardizzati tramite mapping',
        'Impact': 'Variabile'
    },
    {
        'Issue': 'Canale Marketing mancante',
        'Status': 'Solved',
        'Action': 'Imputato come "Unknown"',
        'Impact': f"{data_copy['MARKETING_CHANNEL'].value_counts().get('Unknown', 0)} righe"
    },
    {
        'Issue': 'Spedizione antecedente acquisto',
        'Status': 'Unsolved',
        'Action': 'Mantenuto per analisi, segnalato',
        'Impact': f"{pct_invalid:.1f}%"
    }
]

log_df = pd.DataFrame(issues_log)
print(log_df)

# Output finale pulito
print("\nDataset Finale:")
data_copy.head()


--- STEP N: Note & Document ---
                             Issue    Status  \
0       Formati data inconsistenti    Solved   
1     Nomi prodotti errati (typos)    Solved   
2        Canale Marketing mancante    Solved   
3  Spedizione antecedente acquisto  Unsolved   

                             Action     Impact  
0    Convertito a datetime ISO 8601  100% rows  
1    Standardizzati tramite mapping  Variabile  
2           Imputato come "Unknown"    0 righe  
3  Mantenuto per analisi, segnalato       9.1%  

Dataset Finale:


,USER_ID,ORDER_ID,PURCHASE_TS,SHIP_TS,PRODUCT_NAME,PRODUCT_ID,USD_PRICE,PURCHASE_PLATFORM,MARKETING_CHANNEL,ACCOUNT_CREATION_METHOD,COUNTRY_CODE
0,2c06175e,0001328c3c220830,2020-12-24,2020-12-13,Nintendo Switch,e682,168.0,website,affiliate,unknown,US
1,ee8e5bc2,0002af7a5c6100772,2020-10-01,2020-09-21,Nintendo Switch,e682,160.61,website,direct,desktop,DE
2,9eb4efe0,0002b8350e167074,2020-04-21,2020-02-16,Nintendo Switch,8d0d,151.2,website,direct,desktop,US
3,cac7cbaf,0006d06b98385729,2020-04-07,2020-04-04,Sony PlayStation 5 Bundle,54ed,1132.82,website,direct,desktop,AU
4,6b0230bc,00097279a2f46150,2020-11-24,2020-08-02,Nintendo Switch,8d0d,33.89,website,direct,desktop,TR
